In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.io.wavfile
from IPython.display import Audio

$\textbf{Exercise 1: Initialization}$

In [131]:
#Part(a): Importing the files 

sound1 = np.loadtxt('sounds/sound1.dat')
sound2 = np.loadtxt('sounds/sound2.dat')

sounds = np.zeros((2,sound1.shape[0]))
sounds[0,:], sounds[1,:] = sound1, sound2

#Part(b): Create mixing matrix

A = np.random.rand(2,2)
X_unshuf = A @ sounds

#Part(c): Shuffle the data

np.random.shuffle(X_unshuf.T)
X = X_unshuf

#Part(d): Calculate the correlations between the sources and 
# the mixtures:

rho = np.zeros((2,2))
i = 0

while i < 2:
    j = 0
    while j < 2:
        
        rho[i,j] = np.cov(sounds[i,:], X[j,:])[0,1] #take random element off main diag.
        rho[i,j] /= np.std(sounds[i,:]) * np.std(X[j,:])
        
        j += 1
    i += 1

#Part(e): Center the data matrix X:

X = (X.T - X.mean(axis=1)).T

#Part(f): Initialize unmixing matrix and check for invertibility

while True:
    
    unmix = np.random.rand(2,2)
    
    if isinstance(np.linalg.inv(unmix), np.ndarray):
        
        break
        
    else:
        
        continue


print(unmix)

[[0.4202536  0.3971762 ]
 [0.57371975 0.40892591]]


$\textbf{Exercise 2: Optimization}$

In [184]:
#Part(a): Standard gradient descent:

ffunc = lambda x: 1/(1 + np.exp(-x))
ffunc1 = lambda x: ffunc(x) * (1 - ffunc(x))
ffunc2 = lambda x: ffunc1(x) * (1 - 2 * ffunc(x))

np.vectorize(ffunc)
np.vectorize(ffunc1)
np.vectorize(ffunc2)

def stGrad(eps, unmix, data):
    
    """
    Performs standard gradient descent using logistic activation function
    @params:
    eps: learning rate
    unmix: the unmixing (weight) matrix
    data: the training data
    
    """
    
    W = unmix.copy()
    Wref = np.ones((2,2)) * np.inf
    result = np.zeros((2,2,X.shape[1]))
    dist = np.zeros(X.shape[1])
    
    #Count data iterations:
    iters = 0
        
    for sample in data.T:
        
        iters += 1
        
        if np.linalg.norm(W-Wref) > 10e-20: #cut-off criterion: changes in W are small
        
            #Calculate matrix from second summand:
            vec = ffunc2(W @ sample) / ffunc1(W @ sample)
            mat = np.vstack((vec,vec)).T
            #Calculate dW:
            deltaW = np.linalg.inv(W).T + mat * np.vstack((sample,sample)).T
        
            
            Wref = W 
            W = W + eps*deltaW
            eps *= .999
        
        else:
            
            break
        
    return Wref, iters

WST = stGrad(.001, unmix, X_unshuf)
WSTshuf = stGrad(.001, unmix, X)
WSTshuf

(array([[-1.27378917e-03,  1.32330487e+00],
        [ 1.33585824e+00, -1.73626525e-01]]), 18000)

In [193]:
def natGrad(eps, unmix, data,iters):
    
    """
    Performs natural gradient descent using logistic activation function
    @params:
    eps: learning rate
    unmix: the unmixing (weight) matrix 
    
    """
    iterations = iters
    W = unmix.copy()
    Wref = np.ones((2,2)) * np.inf
    result = np.zeros((2,2,X.shape[1]))
    dist = np.zeros(X.shape[1])
    
    #Count data iterations:
    
    for sample in data.T:
        
        iterations+=1
        
        if np.linalg.norm(W-Wref) > 10e-20: #cut-off criterion: changes in W are small
        
            #Calculate matrix from second summand:
            vec = ffunc2(W @ sample) / ffunc1(W @ sample)
            #Calculate dW:
            temp = np.eye(W.shape[0]) + vec @ (W @ sample)
            deltaW = temp @ W
                                                        
            Wref = W 
            W = W + eps*deltaW
            eps *= .999
        
        else:
            
            break
            
    if np.linalg.norm(W-Wref) > 10e-20:
        
        natGrad(eps,W,data,iters=iterations)
        
    return Wref, iterations 

WNat = natGrad(.001,unmix, X_unshuf,iters=0)[0]
WNatShuf = natGrad(.001,unmix, X,iters=0)[0]
WNatShuf

array([[0.28894839, 0.38748428],
       [0.70600835, 0.41941534]])

$\textbf{Part[3]: Results:}$


In [127]:
#Birdsong:
Audio(sound1, rate=8192)

In [128]:
#Händel:
Audio(sound2, rate=8192)

In [134]:
#Unshuffled mixed sources: birdsong
Audio(X_unshuf[0,:], rate=8192)

In [136]:
#After permutation:
Audio(X[0,:], rate=8192)

In [135]:
#Unshuffled mixed sources: Händel
Audio(X_unshuf[1,:], rate=8192)

In [138]:
#After permutation:
Audio(X[1,:], rate=8192)

In [168]:
#Shuffled birdsong estimate (standard gradient)
Audio((WSTshuf @ X)[0,:], rate=8192)

In [174]:
#Shuffled Händel estimate (natural gradient)
Audio((WNatShuf @ X)[1,:], rate=8192)

Oh my God, these are terrible. Why don't they recover the original sources? Is something wrong with the gradient descent methods? I think it maybe the cut-criterion, because the algorithm is running very few iterations

In [180]:
estimate1 = (WSTshuf @ X)[0,:]
estimate2 = (WSTshuf @ X)[1,:]

print(np.cov(sound1, estimate1) / (np.std(sound1) * np.std(estimate1)))
print(np.cov(sound1, estimate2) / (np.std(sound2) * np.std(estimate1)))

[[ 6.51175429e-01 -1.14946047e-03]
 [-1.14946047e-03  1.53585513e+00]]
[[ 0.65097437 -0.00216194]
 [-0.00216194  0.25270997]]


Again, the correlations are terribly small. Something is very wrong here.